# 컴피티션 개인전
- 17시30분 마감
- 평가지표: AUC
- 1~3등에게 스타벅스 쿠폰을 전달

In [170]:
%pip install catboost

In [171]:
%pip install optuna

In [172]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import optuna

In [173]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 경로와 시드값

In [174]:
DATA_PATH  = "/content/drive/MyDrive/the_datas/data/"
SEED = 42

# 데이터 불러오기

In [175]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터
train_tr.shape ,test_tr.shape, train_target.shape, submit.shape

((523105, 7), (441196, 7), (14940, 2), (12225, 2))

In [176]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_3.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_3.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 435), (12225, 435))

In [177]:
# display(train.head())
# display(train.info())

- 공통 피처 파일 불러오기

In [178]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_3.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_3.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 435), (12225, 435))

# 결측치 처리

In [179]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

,0
3회이상count,4214
구매금액표준편차,388
구매금액왜도,840
구매금액첨도,1265


In [180]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

,0
3회이상count,3284
구매금액표준편차,242
구매금액왜도,599
구매금액첨도,954


In [181]:
train_ft["3회이상count"] = train_ft["3회이상count"].fillna(0)
train_ft["구매금액표준편차"] = train_ft["구매금액표준편차"].fillna(0)
train_ft["구매금액왜도"] = train_ft["구매금액왜도"].fillna(0)
train_ft["구매금액첨도"] = train_ft["구매금액첨도"].fillna(0)

test_ft["3회이상count"] = test_ft["3회이상count"].fillna(0)
test_ft["구매금액표준편차"] = test_ft["구매금액표준편차"].fillna(0)
test_ft["구매금액왜도"] = test_ft["구매금액왜도"].fillna(0)
test_ft["구매금액첨도"] = test_ft["구매금액첨도"].fillna(0)

In [182]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [183]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 434), (12225, 434))

- 추가 피처 만들어 보기

In [184]:
train_ft

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,시간별_구매횟수비율_15,시간별_구매횟수비율_16,시간별_구매횟수비율_17,시간별_구매횟수비율_18,시간별_구매횟수비율_19,시간별_구매횟수비율_20,시간별_구매횟수비율_21,시간별_구매횟수비율_22,최대구매액_대분류,최소구매액_대분류
0,16,11,23,0.050000,0.250000,0.400000,0.300000,3,0.300000,0.450000,...,0.125000,0.062500,0.187500,0.250000,0.062500,0.000,0.0,0.0,남성정장스포츠,가정용품
1,41,21,16,0.357143,0.166667,0.357143,0.119048,3,0.214286,0.476190,...,0.000000,0.073171,0.121951,0.219512,0.097561,0.000,0.0,0.0,여성캐주얼,공산품
2,102,56,6,0.464912,0.140351,0.175439,0.219298,0,0.122807,0.728070,...,1.368421,1.157895,0.578947,0.684211,0.210526,0.000,0.0,0.0,명품잡화,명품잡화
3,191,92,3,0.379147,0.180095,0.236967,0.203791,3,0.161137,0.616114,...,1.550000,1.300000,1.850000,1.200000,1.100000,0.050,0.0,0.0,여성의류파트,케주얼_구두_아동
4,55,27,11,0.112903,0.612903,0.209677,0.064516,4,0.096774,0.741935,...,0.285714,0.952381,0.238095,0.190476,0.285714,0.000,0.0,0.0,명품잡화,공산품파트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,16,10,24,0.000000,0.578947,0.052632,0.368421,2,0.263158,0.578947,...,0.300000,0.100000,0.300000,0.100000,0.200000,0.000,0.0,0.0,잡화,영플라자
14936,8,8,29,0.125000,0.625000,0.250000,0.000000,0,0.000000,0.500000,...,0.019231,0.019231,0.038462,0.038462,0.038462,0.000,0.0,0.0,여성의류파트,잡화파트
14937,22,12,26,0.208333,0.458333,0.208333,0.125000,1,0.375000,0.541667,...,0.111111,0.111111,0.044444,0.022222,0.022222,0.000,0.0,0.0,영플라자,공산품
14938,31,12,28,0.393939,0.393939,0.212121,0.000000,3,0.151515,0.484848,...,0.050000,0.025000,0.075000,0.025000,0.150000,0.125,0.0,0.0,영플라자,영플라자


In [185]:
cols = [ col for col in train_ft.columns if col.startswith("중_pivot_횟수비율_") ]

In [186]:
cols

['중_pivot_횟수비율_DC캐주얼',
 '중_pivot_횟수비율_GBR  지원',
 '중_pivot_횟수비율_L_B침구',
 '중_pivot_횟수비율_NB제화',
 '중_pivot_횟수비율_NB핸드백',
 '중_pivot_횟수비율_N_B침구',
 '중_pivot_횟수비율_TOP디자이너',
 '중_pivot_횟수비율_TV.VTR',
 '중_pivot_횟수비율_TV_VCR',
 '중_pivot_횟수비율_TV_VTR',
 '중_pivot_횟수비율_가방',
 '중_pivot_횟수비율_가스기기',
 '중_pivot_횟수비율_가스렌지',
 '중_pivot_횟수비율_가전특정',
 '중_pivot_횟수비율_가정잡화',
 '중_pivot_횟수비율_건강식품',
 '중_pivot_횟수비율_건강용품',
 '중_pivot_횟수비율_건식품',
 '중_pivot_횟수비율_건어물',
 '중_pivot_횟수비율_곡물',
 '중_pivot_횟수비율_골프(LC)',
 '중_pivot_횟수비율_골프(NB)',
 '중_pivot_횟수비율_골프(국내)',
 '중_pivot_횟수비율_골프(단품)',
 '중_pivot_횟수비율_골프(수입)',
 '중_pivot_횟수비율_골프(용품)',
 '중_pivot_횟수비율_골프단품',
 '중_pivot_횟수비율_골프용품',
 '중_pivot_횟수비율_골프웨어',
 '중_pivot_횟수비율_과자',
 '중_pivot_횟수비율_과자류',
 '중_pivot_횟수비율_교복행사',
 '중_pivot_횟수비율_구두수선',
 '중_pivot_횟수비율_구두임대',
 '중_pivot_횟수비율_구두행사',
 '중_pivot_횟수비율_국내부띠끄',
 '중_pivot_횟수비율_국내종합화장품',
 '중_pivot_횟수비율_국내화장품',
 '중_pivot_횟수비율_국산화장품',
 '중_pivot_횟수비율_그라스',
 '중_pivot_횟수비율_기타식품',
 '중_pivot_횟수비율_남성구두',
 '중_pivot_횟수비율_남성잡화',
 '중_pivot_횟수비율_내셔날',
 '중_pivot

In [187]:
train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 437), (12225, 437))

In [188]:
# 최소 구매금액범주화
# 중분류 개수

## Feature Encoding

In [189]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
최대구매액_대분류,28
최소구매액_대분류,28


In [190]:
%pip install category_encoders

In [191]:
import category_encoders as ce

In [192]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["주구매지점"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["주구매지점"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 441), (12225, 441))

In [193]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최소구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최소구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 469), (12225, 469))

In [194]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[["최대구매액_대분류"]])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[["최대구매액_대분류"]])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 497), (12225, 497))

In [195]:
# enc = ce.count.CountEncoder()
# train_ft["최소구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최소구매액_대분류"]])
# test_ft["최소구매액_대분류_cnt"] = enc.transform(test_ft[["최소구매액_대분류"]])

# train_ft.shape, test_ft.shape

In [196]:
# enc = ce.count.CountEncoder()
# train_ft["최대구매액_대분류_cnt"] = enc.fit_transform(train_ft[["최대구매액_대분류"]])
# test_ft["최대구매액_대분류_cnt"] = enc.transform(test_ft[["최대구매액_대분류"]])

# train_ft.shape, test_ft.shape

- 문자열 피처 삭제

In [197]:
cols

['주구매지점', '최대구매액_대분류', '최소구매액_대분류']

In [198]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 494), (12225, 494))

In [199]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [200]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [201]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,구매횟수,내점일수,구매주기,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,12시이전구매비율,12시이후_18시이전구매비율,...,최대구매액_대분류_19,최대구매액_대분류_20,최대구매액_대분류_21,최대구매액_대분류_22,최대구매액_대분류_23,최대구매액_대분류_24,최대구매액_대분류_25,최대구매액_대분류_26,최대구매액_대분류_27,최대구매액_대분류_28
0,-0.437126,-0.369867,0.002987,-1.029777,0.001191,0.838272,0.338186,0.109631,0.718557,-0.654150,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
1,0.239394,0.144110,-0.356452,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.269465,-0.542415,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
2,1.890101,1.943028,-0.869935,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,-0.209830,0.532172,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
3,4.298510,3.793345,-1.023980,0.420933,-0.327474,0.008592,-0.135636,0.109631,-0.009001,0.054536,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497
4,0.618244,0.452496,-0.613193,-0.752532,1.707410,-0.130285,-0.821561,0.693965,-0.346226,0.591326,...,-0.193649,-0.131248,-0.07785,-0.1402,-0.200973,-0.090737,-0.071977,-0.146044,-0.067118,-0.050497


# 정답 데이터

In [202]:
target = train_target["target"]
target

,target
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0
...,...
14935,0.0
14936,0.0
14937,0.0
14938,1.0


# 모델 튜닝하기

In [203]:
class RF_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
              'n_estimators': trial.suggest_int('n_estimators', 100, 1001),
              'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
              'max_depth': trial.suggest_int('max_depth', 3, 41),
              'min_samples_split': trial.suggest_int('min_samples_split', 6, 30),
              'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
        }
        model = RandomForestClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class XG_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50,201),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'gamma': trial.suggest_float('gamma', 0.1, 5.1, step=0.1),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1)
            }
        model = XGBClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LR_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'C': trial.suggest_float('C', 0.001, 10, step=0.001),
            'penalty': trial.suggest_categorical('penalty', ['l2',  None]),
            }
        model = LogisticRegression(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

class LGB_Objective:
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = KFold(n_splits=5, shuffle=True, random_state=self.seed)

    def __call__(self, trial):
        hp = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 501),
            'learning_rate': trial.suggest_float('learning_rate',0.01, 0.31, step=0.01),
            'max_depth': trial.suggest_int('max_depth', 3, 16),
            'num_leaves': trial.suggest_int('num_leaves', 20,101),
            'min_child_samples': trial.suggest_int('min_child_samples', 5,31),
            }
        model = LGBMClassifier(random_state=self.seed, **hp)
        scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
        return scores.mean()

In [204]:
sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델

In [82]:
rf_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-12 01:43:20,166] A new study created in memory with name: no-name-6e4b6c73-03f3-48f3-9579-f80ef1b30ba3


In [84]:
rf_objective_func = RF_Objective(train_ft, target, SEED)
rf_study.optimize(rf_objective_func, n_trials=10)

[W 2024-11-12 01:52:21,050] Trial 1 failed with parameters: {'n_estimators': 642, 'criterion': 'gini', 'max_depth': 40, 'min_samples_split': 26, 'max_features': 'sqrt'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-80-ad2f7da7b6a5>", line 17, in __call__
    scores = cross_val_score(model, self.x, self.y, cv=self.cv, scoring='f1_macro', n_jobs=-1)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 712, in cross_val_score
    cv_results = cross_validate(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib

KeyboardInterrupt: 

In [ ]:
rf_study.best_value, rf_study.best_trial.params

In [205]:
lr_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-12 03:39:51,029] A new study created in memory with name: no-name-2a7d4960-6c12-4834-9527-504f0f1aa33c


In [206]:
mask = train_ft.isna().sum() > 0
train_ft.isna().sum()[mask]

,0


In [207]:
lr_objective_func = LR_Objective(train_ft, target, SEED)
lr_study.optimize(lr_objective_func, n_trials=200)

[I 2024-11-12 03:40:04,053] Trial 0 finished with value: 0.690972824354945 and parameters: {'C': 3.746, 'penalty': 'l2'}. Best is trial 0 with value: 0.690972824354945.
[I 2024-11-12 03:40:11,672] Trial 1 finished with value: 0.6911569197568891 and parameters: {'C': 5.987, 'penalty': 'l2'}. Best is trial 1 with value: 0.6911569197568891.
[I 2024-11-12 03:40:17,337] Trial 2 finished with value: 0.6914464002374353 and parameters: {'C': 0.581, 'penalty': 'l2'}. Best is trial 2 with value: 0.6914464002374353.
[I 2024-11-12 03:40:25,852] Trial 3 finished with value: 0.6914750490825103 and parameters: {'C': 7.081, 'penalty': None}. Best is trial 3 with value: 0.6914750490825103.
[I 2024-11-12 03:40:31,871] Trial 4 finished with value: 0.6913236145783025 and parameters: {'C': 8.325, 'penalty': 'l2'}. Best is trial 3 with value: 0.6914750490825103.
[I 2024-11-12 03:40:39,971] Trial 5 finished with value: 0.6914750490825103 and parameters: {'C': 1.835, 'penalty': None}. Best is trial 3 with val

In [208]:
lr_study.best_value, lr_study.best_trial.params

(0.6915735318924592, {'C': 5.266, 'penalty': 'l2'})

In [209]:
xg_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

[I 2024-11-12 04:02:24,026] A new study created in memory with name: no-name-a5836b8e-4d5b-418d-8da9-598efbc06633


In [210]:
xg_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

xg_objective_func = XG_Objective(train_ft, target, SEED)
xg_study.optimize(xg_objective_func, n_trials=150)

[I 2024-11-12 04:02:24,038] A new study created in memory with name: no-name-b77e29f5-d540-4568-8ac6-be4bde22fc72
[I 2024-11-12 04:02:50,701] Trial 0 finished with value: 0.7009250730304407 and parameters: {'n_estimators': 142, 'learning_rate': 0.060000000000000005, 'max_depth': 3, 'gamma': 4.9, 'subsample': 1.0, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.7009250730304407.
[I 2024-11-12 04:04:06,528] Trial 1 finished with value: 0.7081691487332437 and parameters: {'n_estimators': 96, 'learning_rate': 0.04, 'max_depth': 9, 'gamma': 2.3000000000000003, 'subsample': 0.5, 'colsample_bytree': 0.7}. Best is trial 1 with value: 0.7081691487332437.
[I 2024-11-12 04:04:25,121] Trial 2 finished with value: 0.70402063728328 and parameters: {'n_estimators': 55, 'learning_rate': 0.29000000000000004, 'max_depth': 5, 'gamma': 3.4000000000000004, 'subsample': 0.6, 'colsample_bytree': 0.8}. Best is trial 1 with value: 0.7081691487332437.
[I 2024-11-12 04:06:04,122] Trial 3 finished with va

In [211]:
xg_study.best_value, xg_study.best_trial.params

(0.7224812968364616,
 {'n_estimators': 196,
  'learning_rate': 0.06999999999999999,
  'max_depth': 5,
  'gamma': 2.5000000000000004,
  'subsample': 0.6,
  'colsample_bytree': 0.9})

In [39]:
xg_study.best_trial.params = {'n_estimators': 193,
  'learning_rate': 0.15000000000000002,
  'max_depth': 3,
  'gamma': 2.6,
  'subsample': 0.7,
  'colsample_bytree': 0.6}

ValueError: No trials are completed yet.

In [212]:
lgb_study = optuna.create_study(
    direction='maximize',
    sampler= sampler
)

lgb_objective_func = LGB_Objective(train_ft, target, SEED)
lgb_study.optimize(lgb_objective_func, n_trials=100)

[I 2024-11-12 05:54:21,419] A new study created in memory with name: no-name-c0abc9a8-a96a-4d40-a6ad-41a85525f2e2
[I 2024-11-12 05:54:52,698] Trial 0 finished with value: 0.7006614019271852 and parameters: {'n_estimators': 104, 'learning_rate': 0.23, 'max_depth': 13, 'num_leaves': 66, 'min_child_samples': 25}. Best is trial 0 with value: 0.7006614019271852.
[I 2024-11-12 05:55:26,920] Trial 1 finished with value: 0.7058458902862511 and parameters: {'n_estimators': 273, 'learning_rate': 0.17, 'max_depth': 8, 'num_leaves': 22, 'min_child_samples': 7}. Best is trial 1 with value: 0.7058458902862511.
[I 2024-11-12 05:55:41,761] Trial 2 finished with value: 0.7062640873866695 and parameters: {'n_estimators': 64, 'learning_rate': 0.2, 'max_depth': 7, 'num_leaves': 61, 'min_child_samples': 29}. Best is trial 2 with value: 0.7062640873866695.
[I 2024-11-12 05:56:16,092] Trial 3 finished with value: 0.710247175137851 and parameters: {'n_estimators': 162, 'learning_rate': 0.13, 'max_depth': 13, 

In [215]:
lgb_study.best_value, lgb_study.best_trial.params

(0.719299200944115,
 {'n_estimators': 278,
  'learning_rate': 0.03,
  'max_depth': 14,
  'num_leaves': 40,
  'min_child_samples': 24})

In [ ]:
# (0.8977595290050033,
#  {'n_estimators': 422,
#   'learning_rate': 0.03,
#   'max_depth': 12,
#   'num_leaves': 50,
#   'min_child_samples': 30,
#   'subsample': 0.8,
#   'colsample_bytree': 0.5,
#   'reg_alpha': 0.8,
#   'reg_lambda': 1.0})

# cv 점수 확인하기

In [216]:
cv = KFold(5, random_state=SEED, shuffle=True)

In [217]:
models = [
    ('lr', LogisticRegression(random_state=SEED, **lr_study.best_trial.params)),
    #('rf', RandomForestClassifier(random_state=SEED, **rf_study.best_trial.params)),
    ('xg', XGBClassifier(random_state=SEED, **xg_study.best_trial.params)),
    ('lgb', LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params))
    ]

In [218]:
model_score = []

for model_name, model in models:
    scores = cross_val_score(model, train_ft, target, cv=cv, n_jobs=-1, scoring="f1_macro")
    model_score.append((model_name, scores.mean()))

In [219]:
model_score

[('lr', 0.6915735318924592),
 ('xg', 0.7224812968364616),
 ('lgb', 0.719299200944115)]

In [ ]:
model_score

# 모델 학습및 테스트 데이터 예측

In [40]:
from xgboost import XGBClassifier

model = XGBClassifier(
    random_state=SEED,
    n_estimators=193,
    learning_rate=0.15,
    max_depth=3,
    gamma=2.6,
    subsample=0.7,
    colsample_bytree=0.6
)


- xg

In [220]:
model_xg = XGBClassifier(random_state=SEED, **xg_study.best_trial.params)

In [221]:
model_xg.fit(train_ft, target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2.5000000000000004, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06999999999999999,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=196, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [227]:
pred_proba_xg = model_xg.predict_proba(test_ft)[:,1]
pred_proba_xg.shape

(12225,)

In [228]:
pred_xg = model_xg.predict(test_ft)
pred_xg

array([0, 0, 0, ..., 0, 0, 0])

In [230]:
test_ft.shape

(12225, 494)

In [232]:
pred_xg_train = model_xg.predict(train_ft)
pred_xg_train

array([0, 1, 0, ..., 0, 1, 1])

In [233]:
from sklearn.metrics import f1_score
f1_score(target, pred_xg_train, average='macro')

0.8242195210653409

- lgb

In [234]:
model_lgb = LGBMClassifier(random_state=SEED, **lgb_study.best_trial.params)

In [235]:
model_lgb.fit(train_ft, target)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5874, number of negative: 9066
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.072817 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 54952
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 470
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.393173 -> initscore=-0.433995
[LightGBM] [Info] Start training from score -0.433995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(learning_rate=0.03, max_depth=14, min_child_samples=24,
               n_estimators=278, num_leaves=40, random_state=42)

In [236]:
pred_proba_lgb = model_lgb.predict_proba(test_ft)[:,1]
pred_proba_lgb.shape

(12225,)

In [237]:
pred_lgb = model_lgb.predict(test_ft)
pred_lgb

array([0., 0., 0., ..., 0., 0., 0.])

In [238]:
test_ft.shape

(12225, 494)

In [239]:
pred_lgb_train = model_lgb.predict(train_ft)
pred_lgb_train

array([0., 1., 0., ..., 0., 1., 1.])

In [240]:
from sklearn.metrics import f1_score
f1_score(target, pred_lgb_train, average='macro')

0.8578391251839137

In [241]:
#앙상블
pred = pred_proba_xg * 0.5 + pred_proba_lgb * 0.5
pred

array([0.10219475, 0.43720395, 0.326997  , ..., 0.06162459, 0.19311939,
       0.0222828 ])

In [244]:
pred_final = (pred > 0.5).astype(int)
pred_final.sum()

4089

# 예측 파일 저장후 컴피티션 페이지에 제출

In [246]:
submit

,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5
...,...,...
12220,test_12220,0.5
12221,test_12221,0.5
12222,test_12222,0.5
12223,test_12223,0.5


In [251]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.102195
1,test_1,0.437204
2,test_2,0.326997
3,test_3,0.821938
4,test_4,0.655256
...,...,...
12220,test_12220,0.778495
12221,test_12221,0.298809
12222,test_12222,0.061625
12223,test_12223,0.193119


#업로드

In [252]:
submit.to_csv(f"{DATA_PATH}submit_새출발_튜닝_2_proba.csv",index=False)